## Part 1: Preliminary Code 

Here we've got a handy cell containing the pip installs of some of the more uncommon modules that you'll need to make this work.

In [1]:
!pip install folium
!pip install tabulate
!pip install ipywidgets

Here we import all of the modules and such that are needed to ensure that this notebook runs correctly. If you don't already have it installed, you will need Folium, ipywidgets, and tabulate. If you run the cell above this one it will automatically install all of these.

In [2]:
import ipywidgets as widgets
import pandas as pd
import numpy as np
import folium
from IPython.display import display, clear_output
from tabulate import tabulate
from folium.plugins import MarkerCluster

state_geo = r'./us-states.json' #reads in the location data for the state boundaries
state_data = pd.read_csv('State_ClimateData.csv') #reads in the climate data we collected for the different states (US territories/outlying regions not included)
marker_data= pd.read_fwf('marker_longlat.txt') #like pd.read_csv but for files with fixed width formatted lines, reads in marker locations (capitals!)
marker_data.columns= ['State', 'Latitude', "Longitude"] #adds in column names

Here we do some shenanigans with the marker data set. Ignore this for now- it'll come in handy later when we start displaying data. 

In [3]:
#masks the state data for just the climate categories (still in alphabetical order by state)

state= state_data['State']
temp_marker= state_data['Avg_tempF']
rain_marker=  state_data['Avg_Rain_In']
snow_marker=  state_data['Avg_Snow_In']
humid_marker=  state_data['Avg_Humid']
sunny_marker=  state_data['Sunny_Days']

#------------------------------------------------------------------------------------------

#masks the state dataset for the temperature data and other datas

temp_data = state_data[['State','ABR','Avg_tempF']] 
rain_data = state_data[['State','ABR','Avg_Rain_In']] 
snow_data = state_data[['State','ABR','Avg_Snow_In']] 
humi_data = state_data[['State','ABR','Avg_Humid']] 
sunn_data = state_data[['State','ABR','Sunny_Days']] 

#------------------------------------------------------------------------------------------

#sorts the temperature data from least to greatest

temp_data_sorted = temp_data.sort_values('Avg_tempF') 
rain_data_sorted = rain_data.sort_values('Avg_Rain_In')
snow_data_sorted = snow_data.sort_values('Avg_Snow_In')
humi_data_sorted = humi_data.sort_values('Avg_Humid')
sunn_data_sorted = sunn_data.sort_values('Sunny_Days')

#------------------------------------------------------------------------------------------

temp_high = temp_data_sorted.iloc[:-4:-1]    #filters the data for the top three results
temp_low = temp_data_sorted.iloc[:3]      #filters the data for the lowest three results
rain_high = rain_data_sorted.iloc[:-4:-1]
rain_low = rain_data_sorted.iloc[:3]
snow_high = snow_data_sorted.iloc[:-4:-1]
snow_low = snow_data_sorted.iloc[:3]
humi_high = humi_data_sorted.iloc[:-4:-1]
humi_low = humi_data_sorted.iloc[:3]
sunn_high = sunn_data_sorted.iloc[:-4:-1]
sunn_low = sunn_data_sorted.iloc[:3]

The cell below does some little marker data stuff for later (latitude and longitudes for marker positions), initializes the blank folium map.

In [4]:
lats= marker_data.iloc[:, 1] #sets latitude data 
lons= marker_data.iloc[:, 2] #sets longitude data

locations = list(zip(lats, lons)) #creates a lost of where the markers should be placed

### Part 2: The Cool Stuff

Here we define a lot of the mapping stuff, and go into making the widget! Also, note that the command line code merely silences the automatic map output of the widget upon using the drop-down menu- it doesn't mess with the redefinition of the variable linked to the dropdown menu.

In [5]:
mydata = {'Temperature Data':('Average Temperature','Avg_tempF','OrRd'), #creates a dictionary that the ipywidgets module can work with
          'Rain Data':('Average Rain (inches)', 'Avg_Rain_In', 'BuPu'),
          'Snow Data':('Average Snow (inches)','Avg_Snow_In', 'PuBu'),
          'Humidity Data':('Average Humidity','Avg_Humid', 'YlOrRd'),
          'Sunshine Data':('Average Sun','Sunny_Days', 'YlOrBr')}

default = 'Temperature Data' #defaults the widget to choose temperature data as an input

w = widgets.Dropdown(               #creates a dropdown menu with the data keys from the dictionary above as options
    options=mydata.keys(),
    value = default,
    description='Data',)

Below is a formatting function we later use in our widget to help with the tabular stuff. It goes through and applies datasets depending on the value chosen in the dropdown menu, as well as some conditional map markers!

In [6]:
def high_low_print(w): 
    
    """Prints highs and lows as filtered by the previous cell for each data type"""
    
    if w == 'Temperature Data': #will be explained further below, when we introduce the widget
        high = 'The highest average temperatures'
        low = 'The lowest average temperatures'
        name_high = temp_high
        name_low = temp_low
        popups= ['{}, Average Temperature: {} &degF'.format(state, temp)  for state in state for temp in temp_marker]
        m.add_child(MarkerCluster(locations=locations, popups=popups))
        return popups
        
    elif w == 'Rain Data':
        high = 'The highest average rainfall states'
        low = 'The lowest average rainfall states'
        name_high = rain_high
        name_low = rain_low
        popups= ['{}, Average Rain (in.): {}'.format(state, rain) for state in state for rain in rain_marker]
        m.add_child(MarkerCluster(locations=locations, popups=popups))
        return popups
        
    elif w == 'Snow Data':
        high = 'The highest average snowfall states'
        low = 'The lowest average snowfall states'
        name_high = snow_high
        name_low = snow_low
        popups= ["{}, Average Snow (in.): {}". format(state, snow) for state in state for snow in snow_marker]
        m.add_child(MarkerCluster(locations=locations, popups=popups))
        return popups
        
    elif w == 'Humidity Data':
        high = 'The highest average humidity states'
        low = 'The lowest average humidity states'
        name_high = humi_high
        name_low = humi_low
        popups= ["{}, Average humidity(%): {}". format(state, humid) for state in state for humid in humid_marker]
        m.add_child(MarkerCluster(locations=locations, popups=popups))
        return popups
        
    elif w == 'Sunshine Data':
        high = 'The sunniest states'
        low = 'The least sunniest states'
        name_high = sunn_high
        name_low = sunn_low
        popups= ['{}, Average Sunny Days (per year): {}'.format(state, sun) for state in state for sun in sunny_marker]
        m.add_child(MarkerCluster(locations=locations, popups=popups))
        return popups
        
    else:
        return print("No! Bad user! Choose a valid category!")
    
    print(high)                            #header
    print(tabulate(name_high)) #pretty prints the high data
    print()                                  #for spacing
    print(low)                           #header
    print(tabulate(name_low)) #and pretty prints the low data

The cells below plot all of the data, clickable markers included, and print out the top and bottom three states for the extremity of the chosen category.

In [7]:
m = folium.Map(location= [48, -102],  tiles= 'Cartodb Positron', zoom_start= 3) #creates a folium map using whatever data option was chosen for the widget

m.choropleth(geo_data= state_geo, data= state_data,
             columns= ['ABR', mydata[w.value][1]], 
             key_on= 'feature.id',
             fill_color= mydata[w.value][2], fill_opacity= 0.7, line_opacity= 0.2,
             legend_name= mydata[w.value][0], smooth_factor= 0.1,highlight= True)

tooltip = 'Click me!' #sets the tooltip for each marker with this instruction (for interactivity!)

high_low_print(w.value)

def on_change(change):
    if change['name'] == 'value' and (change['new'] != change['old']):
        clear_output()
        display(w)
        high_low_print(w.value)
        m = folium.Map(location= [48, -102], zoom_start= 3)
        m.choropleth(geo_data= state_geo, data= state_data,
             columns= ['ABR', mydata[w.value][1]],
             key_on= 'feature.id',
             fill_color= mydata[w.value][2], fill_opacity= 0.7, line_opacity= 0.2,
             legend_name= mydata[w.value][0], smooth_factor=  0.1,highlight= True)
        display(m)
    

C:\Users\Maegan\Anaconda3\lib\site-packages\folium\folium.py:432: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [9]:
display(w) #displays the widget as described above

w.observe(on_change) #calls the function on_change to the widget

display(m) #displays the updates map

Dropdown(description='Data', index=2, options=('Temperature Data', 'Rain Data', 'Snow Data', 'Humidity Data', …